This notebook runs a `Qallse` model in simulation mode and shows how to simply visualize the results using utilities defined in module `hepqpr.qallse.plotting`.

# Import and setup

In [1]:
# import modules
import pandas as pd
import numpy as np

from hepqpr.qallse.plotting import *
from hepqpr.qallse import *
from hepqpr.qallse.dsmaker import create_dataset

# initialise the plotting module in "notebook" mode
set_notebook_mode()

# initialise the logging module
import logging
logging.basicConfig()
fmt = logging.Formatter("%(asctime)s.%(msecs)03d %(levelname)s %(name)s: %(message)s", datefmt='%H:%M:%S')
for handler in logging.getLogger().handlers: handler.setFormatter(fmt)
logging.getLogger('hepqpr').setLevel(logging.DEBUG)

# Define run config

In [2]:
# == DATASET CONFIG

dsmaker_config = dict(
    density=.05, # 1% 
)

# == INPUT CONFIG

# whether or not to add missing doublets to the input
add_missing = True 

# == RUN CONFIG

model_class = QallseD0 # model class to use
extra_config = dict() # configuration arguments overriding the defaults

# Generate the data

In [3]:
import tempfile

tempdir = tempfile.TemporaryDirectory()
print(f'using {tempdir.name}')
metas, path = create_dataset(output_path=tempdir.name, gen_doublets=True, random_seed=240834366, **dsmaker_config)

using /var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmptcme5eb3


11:45:29.860 DEBUG hepqpr.qallse.dsmaker.dsmaker: Loaded 120939 hits from /Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000.
11:45:29.900 DEBUG hepqpr.qallse.dsmaker.dsmaker: Filtered hits from barrel. Remaining hits: 65518.
11:45:29.955 DEBUG hepqpr.qallse.dsmaker.dsmaker: Dropped double hits. Remaining hits: 55446.
11:45:29.990 DEBUG hepqpr.qallse.dsmaker.dsmaker: High Pt hits: 32302/2858
11:45:30.121 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_hits=2858
11:45:30.122 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_tracks=409
11:45:30.122 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_important_tracks=36
11:45:30.123 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_noise=651
11:45:30.124 DEBUG hepqpr.qallse.dsmaker.dsmaker:   random_seed=240834366
11:45:30.125 DEBUG hepqpr.qallse.dsmaker.dsmaker:   time=2019-09-30T11:45:30.121620
11:45:31.270 INFO hepqpr.qallse.dsmaker.dsmaker: Doublets (len=28187) generated in f/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmptcme5eb3/

In [4]:
with open(path + '-meta.json') as f:
    print(f.read())

{
    "num_hits": 2858,
    "num_tracks": 409,
    "num_important_tracks": 36,
    "num_noise": 651,
    "random_seed": 240834366,
    "time": "2019-09-30T11:45:30.121620",
    "params": {
        "input_path": "/Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000",
        "output_path": "/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmptcme5eb3",
        "density": 0.05,
        "high_pt_cut": 1.0,
        "double_hits_ok": false,
        "gen_doublets": true,
        "prefix": null,
        "random_seed": 240834366,
        "phi_bounds": null,
        "min_hits_per_track": 5
    }
}


# Execute the model

## Load the data

In [5]:
# load data
dw = DataWrapper.from_path(path)
doublets = pd.read_csv(path + '-doublets.csv')
if add_missing:
    doublets = dw.add_missing_doublets(doublets)
else:
    p, r, ms = dw.compute_score(doublets)
    print(f'got {len(doublets)}.')
    print(f'  Input precision (%): {p*100:.4f}, recall (%): {r*100:.4f}, missing: {len(ms)}')

got 28187 doublets.
  Input precision (%): 0.9859, recall (%): 98.8848
    New precision (%): 0.9969


## Build the model

In [6]:
%%time

# instantiate qallse
model = model_class(dw, **extra_config)
# build the qubo
model.build_model(doublets=doublets)
Q = model.to_qubo()

11:45:31.889 DEBUG hepqpr.qallse.qallse_d0: using config:
11:45:31.890 DEBUG hepqpr.qallse.qallse_d0:     beamspot_center: (0, 0, 0)
11:45:31.890 DEBUG hepqpr.qallse.qallse_d0:     beamspot_width: 27.5
11:45:31.891 DEBUG hepqpr.qallse.qallse_d0:     cheat: False
11:45:31.892 DEBUG hepqpr.qallse.qallse_d0:     d0_denom: 1.0
11:45:31.893 DEBUG hepqpr.qallse.qallse_d0:     d0_factor: 0.5
11:45:31.894 DEBUG hepqpr.qallse.qallse_d0:     max_layer_span: 2
11:45:31.895 DEBUG hepqpr.qallse.qallse_d0:     min_qplet_path: 2
11:45:31.896 DEBUG hepqpr.qallse.qallse_d0:     num_multiplier: -1
11:45:31.896 DEBUG hepqpr.qallse.qallse_d0:     qplet_max_dcurv: 0.0001
11:45:31.897 DEBUG hepqpr.qallse.qallse_d0:     qplet_max_strength: -0.2
11:45:31.898 DEBUG hepqpr.qallse.qallse_d0:     qubo_bias_weight: 0
11:45:31.899 DEBUG hepqpr.qallse.qallse_d0:     qubo_conflict_strength: 1
11:45:31.900 DEBUG hepqpr.qallse.qallse_d0:     rz_power: 1
11:45:31.901 DEBUG hepqpr.qallse.qallse_d0:     strength_bounds: N

-30.956 -8.22027 -320.613
2.995913001280594
-14.2786 -28.5682 -281.137
2.867124024772592
14.2963 -30.049 -280.66200000000003
2.824273899905607
29.6193 12.0433 -319.279
2.9934496143145872
21.2904 24.1022 -287.126
2.88133180663528
18.6205 26.4759 -280.762
2.852371553350135
23.2221 24.2585 -285.124
2.8309286231946524
-13.2107 31.0187 -291.749
2.8499927630057096
-31.2351 13.2978 -284.07
2.816348426742218
-30.2755 -8.66645 -227.80900000000003
2.670343517046508
-26.4661 -17.5644 -256.415
2.780325472538543
-17.7092 -26.3655 -232.71400000000003
2.6831672169689353
1.5866 -31.5788 -232.366
2.6861696717585115
9.94368 30.9284 -242.282
2.7008918128229134
-2.62706 32.3871 -216.098
2.585953383655738
-2.87525 32.3003 -269.798
2.8105929731745345
-3.09579 32.2232 -248.913
2.731559207901312
-29.7019 -11.8073 -164.342
2.3273675737874293
-29.1408 -13.4121 -160.651
2.300845298434867
-30.9168 -8.33244 -163.579
2.3208834558595584
-32.1006 -4.94669 -172.886
2.3622230897602177
-24.1534 -20.8969 -202.716
2.53906

-11.6737 -71.5987 -91.9318
0.8761367884010977
18.6823 -69.6563 -108.868
1.0674429924963322
40.0528 -58.727 -128.88299999999998
1.262364786158072
62.1723 -36.8431 -110.38
1.0800371919634493
65.3722 -28.1009 -90.2626
0.8772436176119122
71.3195 -5.00185 -121.196
1.1913302609967913
70.6541 10.1238 -115.475
1.1416764618803057
72.4473 2.73749 -115.193
1.1223899006860132
51.5658 49.1023 -114.815
1.1381249724333062
34.3724 63.0767 -94.0592
0.9123836052906319
36.6677 62.2365 -103.494
1.0109446858828546
31.3313 64.1898 -99.3588
0.978778168036993
-5.34829 71.8135 -116.521
1.1418227288921576
-8.25677 71.1073 -123.001
1.2056559926828938
-32.1955 64.4665 -79.0612
0.713096650957314
-38.3413 59.9772 -130.987
1.2779169350965986
-41.6269 59.6942 -91.5434
0.8678902146740386
-63.8456 32.2715 -132.537
1.2850781623981078
-61.8071 37.841 -94.4518
0.9072312141771354
-68.7935 19.9739 -123.639
1.2104014379608463
-71.2239 2.71636 -93.0196
0.9087266203121999
-71.1194 -10.3055 -46.3236
0.019489451269427152
-72.391

-76.7894 86.2151 89.2068
0.28016087359255654
-80.5221 84.4311 127.008
0.7040249507647652
-105.206 50.4705 103.094
0.45377369252460265
-103.086 -51.4145 180.02
1.1046151700417368
-1.91244 -115.134 193.867
1.184065118537182
4.5301 -116.038 152.871
0.9182877250662903
97.7119 -63.0298 151.929
0.9100324155433811
113.686 21.3699 171.623
1.0487502865988327
107.79 43.9588 142.846
0.8402127365507982
102.571 52.2297 160.871
0.9839372312645935
100.616 57.9686 160.506
0.9718518032261098
70.0063 91.9144 153.64
0.9294102572641117
30.8701 111.092 154.07299999999995
0.9347677930530269
5.4035400000000005 115.412 143.66
0.8549609489549997
-61.3494 97.8608 178.58599999999996
1.093204303712081
-72.5592 89.8966 177.84099999999995
1.0884848166304462
-92.9011 68.3012 192.193
1.173408937241269
-92.1016 69.6828 207.349
1.2520176738156379
-109.987 37.6253 162.741
0.9857802844177324
-111.752 26.8319 168.59900000000005
1.0365564453626035
-113.988 17.6098 156.659
0.9526751504168781
-117.656 0.947574 229.099
1.3371

-212.433 -153.94 100.0
-1.327937634049182
-147.81799999999996 -209.671 64.0
-0.7705527439176484
-41.9164 -258.839 88.0
-2.511598780933292
142.149 -217.414 121.6
-0.5981648321114462
187.144 -184.573 58.0
-0.1814822670934446
249.797 -64.191 62.2
-0.5987497112635399
261.511 -3.90891 86.8
-2.7439352032848214
249.588 80.9597 58.0
-0.18954403770827077
242.08900000000003 85.7318 138.4
-0.28877871944179956
218.93 138.563 110.4
-0.868250541178713
201.394 160.126 153.2
-0.10887880755040474
165.571 199.233 74.8
-1.8187353798833492
118.106 234.686 110.8
-0.9027571928934649
96.9423 240.018 155.44
-0.09465634467687273
-28.9095 258.321 88.0
-2.362572972074268
-17.4588 257.501 90.4
-1.9359701242622251
-98.0014 239.051 146.8
-0.19028410056554332
-122.228 225.832 102.0
-1.131428226719101
-180.045 182.195 96.5826
-1.3872857128266745
-236.039 103.539 101.2
-1.1828169565896904
-244.997 76.014 134.8
-0.3384405839553041
-256.406 25.8563 124.8
-0.5155925905111014
-256.467 25.5966 94.4
-1.5673822461757656
-259

...


0 3 0 6
0 3 0 6
3 3 0 6
0 3 0 6
2 3 0 6
0 4 0 6
1 4 0 6
0 3 0 6
0 3 0 6
3 2 0 6
2 3 1 6
0 3 1 6
2 3 1 6
2 3 1 6
2 3 1 6
0 4 1 6
0 3 1 6
0 3 1 6
0 4 1 6
0 3 1 6
0 3 1 6
0 3 1 6
0 3 1 6
0 3 1 6
1 4 1 6
2 3 1 6
1 4 1 6
2 3 1 6
1 2 1 6
3 6 1 6
1 1 1 6
1 3 1 6
1 3 1 6
1 3 1 6
1 3 1 6
2 3 1 6
1 3 1 6
0 3 1 6
0 3 1 6
3 3 1 6
2 3 1 6
0 3 1 6
0 3 1 6
2 3 1 6
0 4 1 6
0 3 1 6
1 4 1 6
0 3 1 6
0 3 1 6
2 3 1 6
2 3 1 6
0 3 1 6
2 3 1 6
0 3 1 6
3 6 1 6
0 4 1 6
2 3 1 6
0 3 1 6
0 3 1 6
0 3 1 6
3 4 1 6
0 4 1 6
3 4 1 6
0 3 1 6
0 3 1 6
3 4 1 6
3 6 1 6
0 3 1 6
0 3 1 6
3 4 1 6
1 2 1 6
0 3 1 6
3 6 1 6
0 3 1 6
0 3 1 6
0 3 1 6
0 3 1 6
0 4 1 6
0 4 1 6
3 3 1 6
0 3 1 6
0 3 1 6
0 3 1 6
3 3 1 6
0 3 1 6
0 3 1 6
0 4 1 6
0 3 1 6
3 3 1 6
2 2 1 6
3 4 1 6
0 3 1 6
1 3 1 6
2 2 1 6
0 3 1 6
2 2 1 6
3 4 1 6
0 3 1 6
0 4 1 6
2 2 1 6
1 3 1 6
2 2 1 6
0 4 1 6
2 2 1 6
0 4 1 6
0 4 1 6
0 3 1 6
3 4 1 6
0 3 1 6
0 4 1 6
2 3 1 6
1 3 1 6
1 3 1 6
3 4 1 6
0 3 1 6
1 4 1 6
0 4 1 6
1 3 1 6
1 4 1 6
2 3 1 6
1 4 1 6
2 3 1 6
2 3 1 6
1 3 1 6
2 3 1 6

11:45:38.818 INFO hepqpr.qallse.qallse_d0: Qubo generated in 5.35s. Size: 2900. Vars: 0, excl. couplers: 2900, incl. couplers: 0



3 3 2 6
0 3 2 6
2 2 2 6
1 4 2 6
2 3 2 6
2 3 2 6
2 3 2 6
1 4 2 6
2 3 2 6
0 3 2 6
2 3 2 6
1 3 2 6
3 2 2 6
1 3 2 6
3 3 2 6
3 3 2 6
3 3 2 6
1 3 2 6
3 3 2 6
3 3 2 6
2 3 2 6
3 2 2 6
3 3 2 6
3 3 2 6
3 2 2 6
2 3 2 6
2 3 2 6
3 6 2 6
1 3 2 6
2 3 2 6
1 3 2 6
3 3 2 6
3 6 2 6
3 3 2 6
2 3 2 6
0 3 2 6
0 3 2 6
3 6 2 6
3 3 2 6
3 3 2 6
1 3 2 6
2 3 2 6
1 3 2 6
3 3 2 6
1 3 2 6
2 3 2 6
1 3 2 6
2 3 2 6
1 3 2 6
2 3 2 6
1 3 2 6
1 3 2 6
2 3 2 6
3 3 2 6
1 3 2 6
3 3 2 6
1 3 2 6
0 3 2 6
2 3 2 6
2 3 2 6
1 3 2 6
2 3 2 6
1 3 2 6
3 3 2 6
3 3 2 6
3 3 2 6
3 3 2 6
1 3 2 6
3 2 2 6
2 3 2 6
3 2 2 6
0 3 2 6
1 3 2 6
0 4 2 6
1 3 2 6
0 3 2 6
2 3 2 6
0 3 2 6
2 3 2 6
0 3 2 6
0 3 2 6
2 3 2 6
2 3 2 6
3 3 2 6
1 3 2 6
1 3 2 6
3 3 2 6
3 3 2 6
2 3 2 6
1 3 2 6
1 3 2 6
3 3 2 6
3 3 2 6
2 3 2 6
3 3 2 6
2 3 2 6
3 2 2 6
3 3 2 6
0 3 2 6
3 3 2 6
0 3 2 6
1 3 2 6
3 3 2 6
0 3 2 6
1 3 2 6
3 6 2 6
0 3 2 6
1 3 2 6
1 3 2 6
1 3 2 6
1 3 2 6
1 3 2 6
2 3 2 6
2 3 2 6
2 3 2 6
2 3 2 6
2 3 2 6
0 3 2 6
2 3 2 6
2 3 2 6
2 3 2 6
2 3 2 6
2 3 2 6
2 3 2 6
0 3 2 6

11:45:38.976 INFO hepqpr.qallse.qallse_d0: Qubo generated in 5.53s. Size: 2970. Vars: 0, excl. couplers: 2955, incl. couplers: 15


3 4 3 6
1 3 3 6
0 3 3 6
3 4 3 6
3 4 3 6
1 2 3 6
0 3 3 6
0 3 3 6
3 4 3 6
2 2 3 6
2 3 3 6
0 3 3 6
3 4 3 6
0 3 3 6
1 2 3 6
1 3 3 6
0 4 3 6
1 3 3 6
0 4 3 6
2 3 3 6
0 3 3 6
2 3 3 6
1 3 3 6
0 3 3 6
1 3 3 6
2 3 3 6
2 3 3 6
1 3 3 6
2 3 3 6
1 3 3 6
2 3 3 6
0 3 3 6
2 3 3 6
2 2 3 6
1 3 3 6
2 3 3 6
2 2 3 6
2 2 3 6
1 3 3 6
2 3 3 6
1 3 3 6
1 3 3 6
2 2 3 6
2 3 3 6
1 3 3 6
0 4 3 6
2 2 3 6
1 3 3 6
1 3 3 6
2 3 3 6
3 3 3 6
1 3 3 6
1 2 3 6
2 2 3 6
0 3 3 6
2 2 3 6
2 3 3 6
0 3 3 6
2 2 3 6
3 3 3 6
2 2 3 6
0 4 3 6
3 3 3 6
3 3 3 6
0 3 3 6
2 3 3 6
1 3 3 6
1 2 3 6
2 3 3 6
0 3 3 6
0 4 3 6
0 3 3 6
2 3 3 6
0 3 3 6
0 3 3 6
0 3 3 6
0 3 3 6
0 3 3 6
0 3 3 6
2 3 3 6
0 3 3 6
1 2 3 6
2 2 3 6
0 3 3 6
2 3 3 6
3 3 3 6
1 4 3 6
2 3 3 6
1 4 3 6
3 3 3 6
3 6 3 6
2 3 3 6
2 3 3 6
1 2 3 6
1 2 3 6
2 3 3 6
1 4 3 6
1 2 3 6
2 3 3 6
3 6 3 6
3 3 3 6
2 3 3 6
3 6 3 6
1 2 3 6
2 3 3 6
3 3 3 6
0 3 3 6
2 2 3 6
0 3 3 6
0 4 3 6
2 2 3 6
0 3 3 6
0 4 3 6
0 3 3 6
1 2 3 6
0 4 3 6
3 4 3 6
3 4 3 6
2 2 3 6
2 3 3 6
2 2 3 6
3 3 3 6
0 3 3 6
0 3 3 6
3 3 3 6


## Sample the QUBO

Here, we use [_qbsolv_](https://github.com/dwavesystems/qbsolv) with default arguments. This means only classical algorithms.

In [7]:
%%time
# execute the qubo
response = model.sample_qubo(Q=Q)

11:45:40.112 INFO hepqpr.qallse.qallse_d0: QUBO of size 2970 sampled in 1.11s (seed 33608734).


CPU times: user 1.1 s, sys: 7.04 ms, total: 1.11 s
Wall time: 1.11 s


## Parse the results

In [8]:
# get all output doublets
all_doublets = model.process_sample(next(response.samples()))
# recreate tracks and resolve remaining conflicts
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

11:45:40.125 INFO hepqpr.qallse.track_recreater: Found 0 conflicting doublets


# Evaluate the results

## Print statistics and scores

In [9]:
# stats about the qbsolv run
en0 = dw.compute_energy(Q)
en = response.record.energy[0]

print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
occs = response.record.num_occurrences
print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

# scores
p, r, missings = dw.compute_score(final_doublets)
print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(missings)}')
trackml_score = dw.compute_trackml_score(final_tracks)
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

SAMPLE -- energy: -164.5309, ideal: -163.9784 (diff: -0.552433)
          best sample occurrence: 38/57
SCORE  -- precision (%): 99.57446808510639, recall (%): 86.98884758364312, missing: 35
          tracks found: 50, trackml score (%): 85.64671840562016


### Plot the results

You have the following at your disposal:
```python
iplot_results(dw, final_doublets, missings, dims=list('xy'))
iplot_results_tracks(dw, final_doublets, missings, dims=list('zr'))
iplot_results_any(dw, dw.real_doublets, dims=list('xy'))
```

For 3D plots, simply define `dims` as 3-dimensional. For example:
```python
iplot_results(dw, final_doublets, missings, dims=list('zxy'))
```

In [10]:
iplot_results(dw, final_doublets, missings, dims=list('xy'))

In [11]:
iplot_results(dw, final_doublets, missings, dims=list('zr'))

In [12]:
iplot_results(dw, final_doublets, missings, dims=list('zxy'))